<div>
<center><img src="https://i.imgur.com/RZnmNC3.gif" width="500" height="500"></center>
<center><img src="https://habrastorage.org/webt/t6/sr/jr/t6srjrmjjmm6qn8gpld9emy4txu.gif" width="500" height="500"></center>
</div>


<div class="alert alert-block alert-info">  
<h1><strong><center>The Simpsons</center></strong></h1>
    <p>The Simpsons is an American animated sitcom created by Matt Groening. The series is a satirical depiction of working-class life, epitomized by the Simpson family, which consists of Homer, Marge, Bart, Lisa, and Maggie. The show is set in the fictional town of Springfield and parodies American culture and society, television, and the human condition.
    </p>
</div>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports

In [ ]:
!pip install NRCLex

In [ ]:
import matplotlib.pyplot as plt
import cufflinks as cf
import plotly
import plotly.express as px
import numpy as np
import re
import nltk
import plotly.graph_objs as go
import random
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg

from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,
                                  AnnotationBbox)
from matplotlib.cbook import get_sample_data
from wordcloud import WordCloud,STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer
from pandas import DataFrame
from PIL import Image
from plotly import tools
from plotly.offline import init_notebook_mode,iplot,plot
from nrclex import NRCLex
from nltk.corpus import stopwords
from colorama import Fore, Back, Style
y_ = Fore.YELLOW
r_ = Fore.RED
g_ = Fore.GREEN
b_ = Fore.BLUE
m_ = Fore.MAGENTA
stop = stopwords.words('english')

In [ ]:
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# A quick look at the data we're working with

In [ ]:
df = pd.read_csv('../input/the-simpsons-dataset/simpsons_script_lines.csv')
df.head(5)

In [ ]:
df = df.head(6000)

In [ ]:
df.shape

In [ ]:
df = df.dropna()

In [ ]:
df['word_count'] = df['word_count'].astype(str).astype(int)

In [ ]:
df.dtypes

# Text preprocessing

In [ ]:
description_list=[]
for description in df['normalized_text']:
    description=re.sub("[^a-zA-Z]", " ", description)
    description=description.lower()
    description=nltk.word_tokenize(description)
    description=[word for word in description if not word in set(stopwords.words("english"))]
    lemma=nltk.WordNetLemmatizer()
    description=[lemma.lemmatize(word) for word in description]
    description=" ".join(description)
    description_list.append(description)
df["normalized_text_new"]=description_list
df.head(5)

In [ ]:
for i,row in df.iterrows():
    print(row['character_id'],row['raw_character_text'])

In [ ]:
val_homer=[]
val_bart=[]
val_marge=[]
val_lisa=[]

for i,row in df.iterrows():
    val = row['normalized_text_new']
    if row['character_id'] == 2:
        val_homer.append(val)
    elif row['character_id']== 8:
        val_bart.append(val)
    elif row['character_id'] == 1:
        val_marge.append(val)
    elif row['character_id']== 9:
        val_lisa.append(val)    


In [ ]:
pat = r'\b(?:{})\b'.format('|'.join(stop))
def text_cleaning(val_list):
    df1 = DataFrame (val_list,columns =['normalized_text_new']).dropna()
    df1["normalized_text_new"] = df1["normalized_text_new"].str.replace(pat, '')
    df1["normalized_text_new"] = df1["normalized_text_new"].str.replace(r'\s+', ' ')
    return df1

In [ ]:
bart = text_cleaning(val_bart)
homer = text_cleaning(val_homer)
marge = text_cleaning(val_marge)
lisa = text_cleaning(val_lisa)

# Word Clouds

In [ ]:
wc = WordCloud()
wc.generate(' '.join(bart['normalized_text_new']))
plt.imshow(wc, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
wc = WordCloud(background_color="white", max_words=2000,
               stopwords=STOPWORDS, max_font_size=256,
               random_state=42, width=500, height=500)
wc.generate(' '.join(bart['normalized_text_new']))
plt.imshow(wc, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
def wordcloud(char,img_path,color):
    plt.subplots(figsize=(10,10))
    mask = np.array(Image.open(img_path))
    wc = WordCloud(stopwords=STOPWORDS, 
                   mask=mask, background_color="white", contour_width=2, contour_color=color,
                   max_words=2000, max_font_size=256,
                   random_state=42, width=mask.shape[1],
                   height=mask.shape[0])
    wc.generate(' '.join(char['normalized_text_new']))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis('off')
    plt.show()

# Unigrams, Bigrams and Trigrams

In [ ]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]


def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
def plot(x,char,name,w,p):
    common_words = x(char['normalized_text_new'], 20)
    common_words_df = DataFrame (common_words,columns=['word','freq'])

    plt.figure(figsize=(16, 10))
    sns.barplot(x='word', y='freq', data=common_words_df,palette=p)
    plt.title("Top 20 "+w+" used by "+name, fontsize=16)
    plt.ylabel("Frequency", fontsize=14)
    plt.yticks(fontsize=13)
    plt.xticks(rotation=90, fontsize=13)
    plt.xlabel("");

In [ ]:
wordcloud(bart,'../input/intermediate-notebooks-data/bart.png',"orange")

In [ ]:
common_words = get_top_n_words(bart['normalized_text_new'], 20)
common_words_df = DataFrame (common_words,columns=['word','freq'])
character_img = mpimg.imread('../input/intermediate-notebooks-data/bart1.jpg')
imagebox = OffsetImage(character_img, zoom=0.07)
xy = (0.5, 0.7)
ab = AnnotationBbox(imagebox, xy,
                    xybox=(17, 37),
                    pad=0.5,frameon=False
                    )
plt.figure(figsize=(16, 6))
ax = sns.barplot(x='word', y='freq', data=common_words_df,palette='OrRd')
ax.add_artist(ab)

plt.title("Top 20 unigrams used by Bart", fontsize=16)
plt.ylabel("Frequency", fontsize=14)
plt.yticks(fontsize=13)
plt.xticks(rotation=45, fontsize=13)
plt.xlabel("");

In [ ]:
plot(get_top_n_bigram,bart,"Bart","bigrams",'OrRd')
plot(get_top_n_trigram,bart,"Bart","trigrams",'OrRd')

In [ ]:
wordcloud(lisa,'../input/intermediate-notebooks-data/lisa.jpg',"orange")

In [ ]:
common_words = get_top_n_words(lisa['normalized_text_new'], 20)
common_words_df = DataFrame (common_words,columns=['word','freq'])
character_img = mpimg.imread('../input/intermediate-notebooks-data/lisa1.jpg')
imagebox = OffsetImage(character_img, zoom=0.4)
xy = (0.5, 0.7)
ab = AnnotationBbox(imagebox, xy,
                    xybox=(16, 30),
                    pad=0.5,frameon=False
                    )
plt.figure(figsize=(16, 6))
ax = sns.barplot(x='word', y='freq', data=common_words_df,palette='PuRd')
ax.add_artist(ab)

plt.title("Top 20 unigrams used by Lisa", fontsize=16)
plt.ylabel("Frequency", fontsize=14)
plt.yticks(fontsize=13)
plt.xticks(rotation=45, fontsize=13)
plt.xlabel("");

In [ ]:
plot(get_top_n_bigram,lisa,"Lisa","bigrams","PuRd")
plot(get_top_n_trigram,lisa,"Lisa","trigrams","PuRd")

In [ ]:
wordcloud(homer,'../input/intermediate-notebooks-data/homer.jpg',"green")

In [ ]:
common_words = get_top_n_words(homer['normalized_text_new'], 20)
common_words_df = DataFrame (common_words,columns=['word','freq'])
character_img = mpimg.imread('../input/intermediate-notebooks-data/homer1.jpg')
imagebox = OffsetImage(character_img, zoom=0.2)
xy = (0.5, 0.7)
ab = AnnotationBbox(imagebox, xy,
                    xybox=(17, 78),
                    pad=0.5,frameon=False
                    )
plt.figure(figsize=(16, 7))
ax = sns.barplot(x='word', y='freq', data=common_words_df,palette='BuPu')
ax.add_artist(ab)

plt.title("Top 20 unigrams used by Homer", fontsize=16)
plt.ylabel("Frequency", fontsize=14)
plt.yticks(fontsize=13)
plt.xticks(rotation=45, fontsize=13)
plt.xlabel("");

In [ ]:
plot(get_top_n_bigram,homer,"Homer","bigrams",'BuPu')
plot(get_top_n_trigram,homer,"Homer","trigrams",'BuPu')

In [ ]:
wordcloud(marge,'../input/intermediate-notebooks-data/marge7.png',"green")

In [ ]:
common_words = get_top_n_words(marge['normalized_text_new'], 20)
common_words_df = DataFrame (common_words,columns=['word','freq'])

character_img = mpimg.imread('../input/intermediate-notebooks-data/marge2.jpg')
imagebox = OffsetImage(character_img, zoom=0.18)
xy = (0.5, 0.7)
ab = AnnotationBbox(imagebox, xy,
                    xybox=(17, 65),
                    pad=0.5,frameon=False
                    )
plt.figure(figsize=(16, 6))
ax = sns.barplot(x='word', y='freq', data=common_words_df,palette='GnBu')
ax.add_artist(ab)

plt.title("Top 20 unigrams used by Marge", fontsize=16)
plt.ylabel("Frequency", fontsize=14)
plt.yticks(fontsize=13)
plt.xticks(rotation=45, fontsize=13)
plt.xlabel("");

In [ ]:
plot(get_top_n_bigram,marge,"Marge","bigrams",'GnBu')
plot(get_top_n_trigram,marge,"Marge","trigrams",'GnBu')

# NRCLex
NRCLex will measure emotional affect from a body of text. Affect dictionary contains approximately 27,000 words, and is based on the National Research Council Canada (NRC) affect lexicon and the NLTK library's WordNet synonym sets.

Emotional affects measured include the following:

* fear
* anger
* anticipation
* trust
* surprise
* positive
* negative
* sadness
* disgust
* joy

In [ ]:
text_object = NRCLex(' '.join(df['normalized_text_new']))

In [ ]:
text_object.affect_frequencies

In [ ]:
text_object.top_emotions

In [ ]:
sentiment_scores = pd.DataFrame(list(text_object.raw_emotion_scores.items())) 

In [ ]:
sentiment_scores = sentiment_scores.rename(columns={0: "Sentiment", 1: "Count"})
sentiment_scores

In [ ]:
fig = px.pie(sentiment_scores, values='Count', names='Sentiment',
             title='Sentiment Scores',
             hover_data=['Sentiment'])
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
sentiment = sentiment_scores['Sentiment'].to_list()
sentiment

In [ ]:
sentiment_words = pd.DataFrame(list(text_object.affect_dict.items()),columns = ['words','sentiments'])
sentiment_words

In [ ]:
for y in sentiment:
    sentiment_words[y] = 9
sentiment_words

In [ ]:
a=0
for i in sentiment_words['sentiments']: 
    for y in sentiment:
        sentiment_words[y][a] = int(y in i)
    a=a+1

In [ ]:
sentiment_words.head(5)

In [ ]:
for y in sentiment:
    word_list = sentiment_words[sentiment_words[y]== 1].words.head(10)
    print(f"{y_}Sentiment:{y}")
    print(f"{r_}{word_list.values}\n")

# Work in progress..
![](https://media.tenor.com/images/c93f6b355513bf7c1b3c415a83323fcd/tenor.gif)